In [2]:
import torch
print(torch.cuda.is_available())  # True라면 GPU 사용 가능
print(torch.cuda.get_device_name(0))  # GPU의 이름 출력

True
NVIDIA GeForce RTX 2080 Ti


In [6]:
import pandas as pd
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data1 = pd.read_csv('data1.csv',encoding='cp949')
data2 = pd.read_csv('data2.csv',encoding='cp949')

data1_test = pd.read_csv('data1_test.csv',encoding='cp949')
data2_test = pd.read_csv('data2_test.csv',encoding='cp949')


data1.head()


,previous_text,current_text,sentiments,situation
0,NaN,엄마 아기가 태어나니까 내가 부모로서 해야 할 게 참 많은 것 같아요.,기쁨,아이가 태어나니 부모가 배우고 익혀야 할 것이 많다.
1,그렇지? 아기 키우는 게 여간 어려운 일이 아니야.,어제 평소보다도 격하게 막 온몸을 써가면서 울더라고요. 얼마나 당황했는지 몰라요.,기쁨,아이가 태어나니 부모가 배우고 익혀야 할 것이 많다.
2,배가 고파서 그랬던 것 아닐까? 아기들은 배가 고프면 몸부림을 친단다.,맞아요. 젖을 물려주니 금세 뚝 그쳤어요. 난 분명히 순했을 것 같은데.,기쁨,아이가 태어나니 부모가 배우고 익혀야 할 것이 많다.
3,말도 마. 네가 얼마나 까탈스러웠는데. 우리 손주가 내 딸을 빼다 박았네.,잠도 못 자고 우는 아이 달래랴 수유하랴 머리카락이 다 빠지는 줄 알았어요.,기쁨,아이가 태어나니 부모가 배우고 익혀야 할 것이 많다.
4,네가 어른 노릇 부모 노릇을 하느라 고생이 정말 많구나.,그러면서 엄마 생각이 많이 났어요. 엄마는 직장까지 다니면서 나를 키우느라 얼마나 ...,기쁨,아이가 태어나니 부모가 배우고 익혀야 할 것이 많다.


In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define constants
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 3

# Sentiment mapping
sentiment_map = {
    '기쁨': 5,
    '당황': 4,
    '상처': 3,
    '불안': 2,
    '슬픔': 1,
    '분노': 0
}

# Initialize the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(sentiment_map))
model.cuda()

# Tokenizer and convert to vector function
def tokenizer_and_convert_to_vector(text, tokenizer, max_len):
    tokens = tokenizer.encode(text, add_special_tokens=True, max_length=max_len, truncation=True)
    return pad_sequences([tokens], maxlen=max_len, dtype="long", value=0, truncating="post", padding="post")[0]

def prepare_data(data, tokenizer, max_len):
    # Combine previous_text and current_text
    combined_texts = data.apply(lambda row: f"{row['current_text']}", axis=1)
    
    # Tokenize using batch_encode_plus for faster processing
    tokens = tokenizer.batch_encode_plus(
        combined_texts.tolist(),  # Convert Series to list
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',  # Pad sequences to the maximum length
        truncation=True,
        return_tensors='pt'  # Directly return PyTorch tensors
    )
    
    # Prepare input ids and attention masks
    input_ids = tokens['input_ids']
    attention_masks = tokens['attention_mask']
    
    # Map sentiments to integer labels
    labels = data['sentiments'].apply(lambda x: sentiment_map.get(x, -1)).values
    labels = torch.tensor(labels, dtype=torch.long)  # Convert to PyTorch tensor

    return input_ids, attention_masks, labels

# 2. 데이터 준비 (prepare_data 이후)
x_train, train_masks, y_train = prepare_data(data1, tokenizer, MAX_LEN)
x_test, test_masks, y_test = prepare_data(data1_test, tokenizer, MAX_LEN)


c:\Users\임석범\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
print(torch.min(y_train))
print(torch.max(y_train))
print(torch.min(y_test))
print(torch.max(y_test))

tensor(0)
tensor(5)
tensor(0)
tensor(5)


In [9]:
# Create DataLoader for training and validation
import random

train_data = TensorDataset(x_train, train_masks, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(x_test, test_masks, y_test)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

optimizer = AdamW(model.parameters(), lr =2e-5, eps = 1e-8)
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer,num_training_steps=0, num_warmup_steps = total_steps)


seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

c:\Users\임석범\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
import torch
import time
import datetime
import numpy as np
from transformers import AdamW, get_linear_schedule_with_warmup
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Early Stopping 설정
class EarlyStopping:
    def __init__(self, patience=3, min_delta=0):
        self.patience = patience  # 개선이 없을 경우 기다리는 에폭 수
        self.min_delta = min_delta  # 최소 변화량
        self.counter = 0  # 개선이 없을 때마다 증가
        self.best_loss = None  # 최저 손실
        self.early_stop = False  # 멈출지 여부

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# EarlyStopping 인스턴스 생성
early_stopping = EarlyStopping(patience=3, min_delta=0.001)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, EPOCHS):
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.    Loss: {:.4f}'.format(step, len(train_dataloader), elapsed, loss.item()))


        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss = 0
    eval_accuracy = 0
    nb_eval_steps = 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        logits = outputs[0]
        label_ids = b_labels.to('cpu').numpy()
        logits = logits.detach().cpu().numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    avg_val_accuracy = eval_accuracy / nb_eval_steps
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

    # Early Stopping 적용
    early_stopping(eval_loss)
    if early_stopping.early_stop:
        print("Early stopping applied!")
        break

print("")
print("Training complete!")



======== Epoch 1 / 3 ========
Training...


c:\Users\임석범\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  Batch    50  of  5,510.    Elapsed: 0:00:31.    Loss: 1.8049
  Batch   100  of  5,510.    Elapsed: 0:01:17.    Loss: 1.8274
  Batch   150  of  5,510.    Elapsed: 0:02:06.    Loss: 1.8056
  Batch   200  of  5,510.    Elapsed: 0:02:56.    Loss: 1.7679
  Batch   250  of  5,510.    Elapsed: 0:03:47.    Loss: 1.7957
  Batch   300  of  5,510.    Elapsed: 0:04:39.    Loss: 1.7746
  Batch   350  of  5,510.    Elapsed: 0:05:31.    Loss: 1.8250
  Batch   400  of  5,510.    Elapsed: 0:06:23.    Loss: 1.7974
  Batch   450  of  5,510.    Elapsed: 0:07:15.    Loss: 1.7929
  Batch   500  of  5,510.    Elapsed: 0:08:07.    Loss: 1.7988
  Batch   550  of  5,510.    Elapsed: 0:08:59.    Loss: 1.7642
  Batch   600  of  5,510.    Elapsed: 0:09:52.    Loss: 1.7747
  Batch   650  of  5,510.    Elapsed: 0:10:44.    Loss: 1.7694
  Batch   700  of  5,510.    Elapsed: 0:11:37.    Loss: 1.7820
  Batch   750  of  5,510.    Elapsed: 0:12:30.    Loss: 1.7725
  Batch   800  of  5,510.    Elapsed: 0:13:22.    Loss:

In [11]:
# 모델 저장하기
torch.save(model.state_dict(), 'roberta_sentiment_model.pt')

In [16]:
# 예측 및 확률 출력 함수
def predict_with_probabilities(text, model, tokenizer, device, max_len=128):
    # 모델을 평가모드로 설정
    model.eval()
    
    # 입력 텍스트를 토큰화 및 벡터화
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 예측 수행
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # 로짓(logit)에서 소프트맥스 함수 적용하여 확률로 변환
    logits = outputs[0]
    probabilities = torch.nn.functional.softmax(logits, dim=1).flatten()

    # 확률을 내림차순으로 정렬하고 순위별로 클래스와 확률 출력
    sorted_indices = torch.argsort(probabilities, descending=True)
    sorted_probabilities = probabilities[sorted_indices]


    
    sentiment_map = {
        5: '기쁨',
        4: '당황',
        3: '상처',
        2: '불안',
        1: '슬픔',
        0: '분노'
    }
    print("예측 확률 순위별 출력:")
    for idx, prob in zip(sorted_indices, sorted_probabilities):
        sentiment = sentiment_map[idx.item()]
        print(f"{sentiment} ({probabilities[idx].item() * 100:.2f} %)")
    
    return sorted_indices, sorted_probabilities

# 예측 함수 사용 예시
text = "오늘은 기분이 정말 좋아요!"
sorted_indices, sorted_probabilities = predict_with_probabilities(text, model, tokenizer, device)

예측 확률 순위별 출력:
기쁨 (percentage: 94.35)
슬픔 (percentage: 1.66)
분노 (percentage: 1.58)
불안 (percentage: 1.22)
당황 (percentage: 0.73)
상처 (percentage: 0.45)


In [7]:
from keras.preprocessing.sequence import pad_sequences
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch


# Create attention masks
# attention_masks = []
# for seq in x_train:
#     print(f"")
#     att_mask = [int(token_id > 0) for token_id in seq]
#     attention_masks.append(att_mask)

# Create attention masks using vectorized operations
x_train_cpu = x_train.cpu()
attention_masks = np.where(x_train_cpu > 0, 1, 0)
attention_masks = torch.tensor(attention_masks, dtype=torch.long).to(device)

# Convert attention masks to torch tensor
    
    
# Convert to torch tensors
x_train = torch.tensor(x_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)


# Split the data into training and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(x_train, y_train, random_state=42, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, x_train, random_state=42, test_size=0.1)

# Create DataLoader for training and validation
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

# Set up the optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * 3  # Assuming 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Print a sample to verify
print("Sample x_train:", x_train[0])
print("Sample y_train:", y_train[0])

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [5]:
# Training function with progress printing and mixed precision training
from tqdm import tqdm

def train_model(model, train_dataloader, validation_dataloader, optimizer, scheduler, device, epochs=EPOCHS, validation_frequency=1):
    model.to(device)  # Move model to GPU
    scaler = torch.cuda.amp.GradScaler()  # For mixed precision training

    for epoch in range(epochs):
        # Training
        model.train()
        total_loss = 0
        
        print(f"\nEpoch {epoch + 1}/{epochs}")
        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Training")

        for step, batch in progress_bar:
            # Unpack the batch and load onto device
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero the gradients
            model.zero_grad()

            # Forward pass with mixed precision
            with torch.cuda.amp.autocast():
                outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                loss = outputs.loss

            # Accumulate the loss
            total_loss += loss.item()

            # Backward pass with mixed precision
            scaler.scale(loss).backward()

            # Clip the norm of the gradients to 1.0 to avoid exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and learning rate
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item()})

        # Calculate the average loss for this epoch
        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average Training Loss: {avg_train_loss}")

        # Validation (run every `validation_frequency` epochs)
        if (epoch + 1) % validation_frequency == 0:
            model.eval()
            eval_loss = 0
            eval_accuracy = 0
            
            print(f"Running Validation... (Epoch {epoch + 1}/{epochs})")
            for batch in tqdm(validation_dataloader, desc="Validation"):
                # Unpack the batch and load onto device
                b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)

                # No gradient calculation during evaluation
                with torch.no_grad():
                    outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                    loss = outputs.loss
                    logits = outputs.logits

                # Accumulate validation loss
                eval_loss += loss.item()

                # Get the predictions and calculate accuracy
                preds = torch.argmax(logits, dim=1).flatten()
                eval_accuracy += (preds == b_labels).cpu().numpy().mean()

            # Calculate the average accuracy for this epoch
            avg_val_accuracy = eval_accuracy / len(validation_dataloader)
            print(f"Validation Accuracy: {avg_val_accuracy}")

# Train the model
train_model(model, train_dataloader, validation_dataloader, optimizer, scheduler, device, epochs=EPOCHS)

C:\Users\임석범\AppData\Local\Temp\ipykernel_12772\2094319600.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # For mixed precision training



Epoch 1/5


Training:   0%|          | 0/2480 [00:00<?, ?it/s]C:\Users\임석범\AppData\Local\Temp\ipykernel_12772\2094319600.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
c:\Users\임석범\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Training:  66%|██████▋   | 1643/2480 [16:32<08:25,  1.65it/s, loss=1.75]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
